In [41]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BinanceDataAPI.settings")
import django
django.setup()
from binance.client import Client
from datetime import datetime

In [38]:
#Binance Client

client = Client('', '')

In [26]:
client.get_order_book(symbol='BNBBTC')

{'lastUpdateId': 259921628,
 'bids': [['0.00386690', '88.97000000'],
  ['0.00386680', '8.43000000'],
  ['0.00386660', '16.85000000'],
  ['0.00386630', '0.87000000'],
  ['0.00386620', '2.87000000'],
  ['0.00386610', '8.83000000'],
  ['0.00386590', '16.00000000'],
  ['0.00386540', '49.88000000'],
  ['0.00386530', '8.32000000'],
  ['0.00386520', '71.00000000'],
  ['0.00386510', '1.10000000'],
  ['0.00386460', '1.10000000'],
  ['0.00386420', '24.14000000'],
  ['0.00386410', '153.69000000'],
  ['0.00386400', '0.54000000'],
  ['0.00386330', '147.91000000'],
  ['0.00386300', '22.55000000'],
  ['0.00386260', '70.66000000'],
  ['0.00386250', '51.72000000'],
  ['0.00386230', '50.65000000'],
  ['0.00386180', '0.55000000'],
  ['0.00386170', '22.60000000'],
  ['0.00386150', '20.00000000'],
  ['0.00386030', '57.08000000'],
  ['0.00386000', '89.05000000'],
  ['0.00385880', '25.65000000'],
  ['0.00385830', '1.22000000'],
  ['0.00385730', '0.33000000'],
  ['0.00385710', '5.49000000'],
  ['0.00385680', 

In [45]:
candles = client.get_klines(symbol='BNBBTC', interval=Client.KLINE_INTERVAL_1MINUTE)

In [49]:
candles[499]

[1557180480000,
 '0.00386630',
 '0.00386630',
 '0.00386630',
 '0.00386630',
 '0.00000000',
 1557180539999,
 '0.00000000',
 0,
 '0.00000000',
 '0.00000000',
 '0']

In [47]:
dt_object = datetime.fromtimestamp(1557150540)

In [48]:
print (dt_object)

2019-05-06 13:49:00


In [50]:
dt_object = datetime.fromtimestamp(1557180480)

In [52]:
print(dt_object)

2019-05-06 22:08:00


In [55]:
dt_object = datetime.fromtimestamp(1557180539)
print(dt_object)

2019-05-06 22:08:59


In [56]:
client.get_server_time()


{'serverTime': 1557181533033}

In [58]:
dt_object = datetime.fromtimestamp(1557181533)
print(dt_object)

2019-05-06 22:25:33


In [59]:
datetime.now()

datetime.datetime(2019, 5, 6, 22, 26, 16, 216817)

In [2]:
User = get_user_model()


In [3]:
print(User)

<class 'django.contrib.auth.models.User'>


In [4]:
User.objects.filter(username='marcus')

<QuerySet [<User: marcus>]>

In [5]:
get_exchange_info()

NameError: name 'get_exchange_info' is not defined

In [ ]:
API_Key = CIdg6PqV0K8RXzqxdcRChPpBKkTzQ0EW9WVCScZvmVkApebxvIpoaJbrLld1hE3F
API_Secret = L2irr6OEYbHzJTO6Iq4CyLnOEA2ZCqxIJSm9uGKNbtOMEGk0D1dxRVSMYm7HXpzG

In [ ]:
if (not User.objects.filter(username=os.environ.get('DJANGO_SUPERUSER_NAME')).exists()):
    User.objects.create_superuser(os.environ.get('DJANGO_SUPERUSER_NAME'), os.environ.get('DJANGO_SUPERUSER_MAIL'), os.environ.get('DJANGO_SUPERUSER_PASS'))
else:
    pass"